### Yuval Lebel 207461567
### Maor Weissmann 031422975
####  https://github.com/yuvalebel/project-yad2

In [66]:
#pip install googlemaps

In [74]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from datetime import datetime
import googlemaps 

In [75]:
# הגדרת ה-API של Google Maps
gmaps = googlemaps.Client(key='$$') 
center_address = "Dizengoff Center, Tel Aviv-Yafo"
BASE_URL = "https://www.ad.co.il/nadlanrent?sp275=17413&sp277=17549,18152"
headers = {"User-Agent": "Mozilla/5.0"}  # התחזות לדפדפן אמיתי
column_mapping = {
    "פרטי הנכס": "property_type",
    "אזור": "Region",
    "עיר": "City",
    "שכונה": "neighborhood",
    "כתובת": "address",
    "קומה": "floor",
    "שטח בנוי": "area",
    "תאריך כניסה": "days_to_enter",
    "חדרים":"room_num",
    "תשלומים בשנה":"num_of_payments",
    "ארנונה בחודש":"monthly_arnona",
    "ועד בית בחודש":"building_tax",
    "על עמודים": "on_pillars",
    "מרוהטת": "is_furnished",
    "מזגן": "ac",
    "חניה": "has_parking",
    "ממ\"ד": "has_safe_room",  # שימוש בגרשיים כפולים
    "מרפסת": "has_balcon",
    "נגישות": "handicap",
    "סורגים": "has_bars",
    "מעלית": "elevator",
    "מחסן": "has_stotsge",
    "מרפסת שמש": "sun_balcony",
    "משופצת": "is_renovated",
    "שטח גינה":"garden_area"
    
}
page = 1
properties = []
previous_count = 0

while True:
    print(f"📄 אוספים נתונים מעמוד {page}...")
    url = f"{BASE_URL}&pageindex={page}" if page > 1 else BASE_URL
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print("❌ שגיאה בטעינת הדף. מפסיקים את האיסוף.")
        break

    soup = BeautifulSoup(response.text, "html.parser")
    ads = soup.find_all("div", class_="card-block")

    if not ads or len(ads) == previous_count:
        print("✅ אין עוד נתונים - סיימנו לאסוף!")
        break

    for ad in ads:
        data = {}
        data["num_of_images"] = int(ad.get("data-images", 0))
        #תמספר תמונות
        card_body = ad.find("div", class_="card-body p-md-3")
        link_tag = card_body.find("a") if card_body else None

        if not link_tag or "href" not in link_tag.attrs:
            print("❌ לא נמצא קישור לדירה. מדלג...")
            continue

        link = "https://www.ad.co.il" + link_tag["href"]
    
        detail_response = requests.get(link, headers=headers)
        if detail_response.status_code != 200:
            print(f"❌ שגיאה בטעינת הדף הפנימי: {link}")
            continue

        detail_soup = BeautifulSoup(detail_response.text, "html.parser")
        rows = detail_soup.find("table", class_="table table-sm mb-4").find_all("tr")
        for row in rows:
            cells = row.find_all("td")
            if len(cells) == 2:  # רק אם יש בדיוק שני תאים בשורה
                key = cells[0].text.strip()
                value = cells[1].text.strip()
                if key in column_mapping:  # בדיקה אם הערך קיים במילון
                    data[column_mapping[key]] = value
        if  "neighborhood" not in data:
            continue 
        # בדיקה אם השכונה היא אחת מהשכונות הרצויות
        if data["neighborhood"] not in ["עג'מי", "פלורנטין"]:
            continue  # אם לא, דלג למודעה הבאה

        if  "address" not in data:
            continue 
        
        #מחיר
        titles = detail_soup.find("div", class_="d-flex justify-content-between").find_all("h2", class_="card-title")
        if len(titles) > 1:
            price_text = titles[1].text.strip()  # שולפים את המחיר כמחרוזת
            price_text = price_text.replace("₪", "").replace(",", "").strip()  # מסירים ₪ ומפרידים אלפים
            data["price"] = float(price_text)  # המרה ל-float
        #כל הפיצ׳רים הבוליאנים
        features = detail_soup.find_all("div", class_="card-icon")
        for feature in features:
            hebrew_name = feature.find("span").text.strip()  # שם התכונה בעברית
            english_name = column_mapping.get(hebrew_name, hebrew_name)  # המרה לאנגלית
            value = 0 if "disabled" in feature["class"] else 1  # 0 אם 'disabled', אחרת 1
            data[english_name] = int(value)
        #data["description"]= " ".join(detail_soup.find("div", class_="p-3").text.split())
       # data["description"] = " ".join([p.text.strip() for p in detail_soup.find("div", class_="p-3").find_all("p", class_="text-word-break")]) 
        paragraphs = [p.text.strip() for p in detail_soup.find("div", class_="p-3").find_all("p", class_="text-word-break")]
        data["description"] = " ".join(sorted(set(paragraphs), key=paragraphs.index))
        # מרחק ממרכז העיר
        if data["address"]:
            #חישוב המרחק מכיכר דיזנגוף
            full_address = f"{data['address']}, תל אביב, ישראל"
            directions_result = gmaps.distance_matrix(full_address, center_address,mode="driving")
            try:
              distance = directions_result["rows"][0]["elements"][0]["distance"]["value"]  /1000# במטרים
              data["distance_from_center"] = distance
            except (KeyError, IndexError):
               data["distance_from_center"] = None  
        
        properties.append(data)

    previous_count = len(ads)
    page += 1
    time.sleep(1)

print(f"✅ סיימנו! נשמרו {len(properties)} דירות.")


📄 אוספים נתונים מעמוד 1...
📄 אוספים נתונים מעמוד 2...
📄 אוספים נתונים מעמוד 3...
📄 אוספים נתונים מעמוד 4...
✅ אין עוד נתונים - סיימנו לאסוף!
✅ סיימנו! נשמרו 52 דירות.


In [69]:
columns = [
    "property_type", "neighborhood", "address", "room_num", "floor", "area", "garden_area","days_to_enter","num_of_payments",
    "monthly_arnona","building_tax","total_floors","description","has_parking","has_stotsge", "elevator", "ac", "handicap", "has_bars",
    "has_safe_room", "has_balcon", "is_furnished", "is_renovated", "price","num_of_images", "distance_from_center"
]
df = pd.DataFrame(properties, columns=columns)
df['floor'] = df['floor'].astype(str)  # המרה למחרוזת
df['floor'] = df['floor'].replace(r'קרקע.*', '0', regex=True)

df[['floor', 'total_floors']] = df['floor'].str.extract(r'(\d+)\s*(?:מתוך\s*(\d+))?')

df[['floor', 'total_floors']] = df[['floor', 'total_floors']].fillna(0).astype(int)  # המרה ל-int ומילוי NaN ב-0
today = datetime.today()


# תאריך נוכחי
today = datetime.today()

# המרת 'כניסה מיידית' ל-0
df.loc[df['days_to_enter'] == 'מיידית', 'days_to_enter'] = 0

# המרת תאריכים למספר ימים עד הכניסה
mask = df['days_to_enter'] != 0


allowed_types = ["דירה", "דירת גן", "בית פרטי/ קוטג'", "גג/פנטהאוז", "מגרשים", "דופלקס", "תיירות ונופש", "כללי"]

# סיווג מחדש של הערכים הלא מתאימים ל-"כללי"
df["property_type"] = df["property_type"].apply(lambda x: x if x in allowed_types else "כללי")
df.loc[:, "garden_area"] = df["garden_area"].fillna(0)
df.loc[:, "days_to_enter"] = df["days_to_enter"].fillna(0)
df["room_num"] = pd.to_numeric(df["room_num"], errors="coerce")



In [73]:
pd.set_option('display.max_columns',None)
df.to_csv("full_rentals.csv", index=False, encoding="utf-8-sig")
df

,property_type,neighborhood,address,room_num,floor,area,garden_area,days_to_enter,num_of_payments,monthly_arnona,building_tax,total_floors,description,has_parking,has_stotsge,elevator,ac,handicap,has_bars,has_safe_room,has_balcon,is_furnished,is_renovated,price,num_of_images,distance_from_center
0,דופלקס,עג'מי,הערמון 9,4.0,3,125,0,0,12,400,500,3,"זה בית מעוצב בצורה מיוחדת, יש אפשרות לשינויים ...",1,1,1,1,0,0,1,1,1,0,12500.0,10,5.124
1,דירה,פלורנטין,לוינסקי 2,3.0,3,70,0,0,12,300,600,10,דירה יפהפיה עם מרפסת מחסן וחנייה בבניין חדש עם...,1,1,1,1,1,0,1,1,1,1,11000.0,10,2.694
2,דירה,עג'מי,שם הגדולים 3,4.0,3,118,0,0,12,450,380,4,דירת 4 חדרים ברחוב שם הגדולים ביפו. דירה גדולה...,1,0,1,1,1,0,1,1,0,1,8000.0,10,6.305
3,דופלקס,עג'מי,קדם 59,5.0,2,170,0,0,12,625,495,5,דופלקס מפואר ומואר- חדש\n4 חדרי שינה לכל חדר ש...,1,0,1,1,1,0,1,1,1,0,15000.0,7,6.652
4,דופלקס,עג'מי,הדגה 8,5.0,3,157,0,0,12,600,490,5,"השכרה, דופלקס, קומה 3, עגמי תל אביב יפו\nדופלק...",1,0,1,1,1,0,1,1,1,0,13000.0,5,5.148
5,כללי,פלורנטין,לוינסקי 9,4.0,3,82,0,0,2,NaN,NaN,8,"ממוקם קרוב לנווה צדק, פארק המסילה (הרכבת הקלה)...",1,0,1,1,0,0,1,1,1,0,8900.0,6,2.626
6,כללי,עג'מי,נמל יפו 32,4.0,3,166,20,0,12,607,100,3,"דירת טריפלקס שלוש קומות ייחודית, \nבבניין של 8...",1,1,1,1,1,1,1,1,1,1,20000.0,7,5.145
7,דירה,פלורנטין,אוריאל אקוסטה 13,1.5,0,27,0,0,12,NaN,80,0,דירה מומלצת ליחיד סטודנט רחוב מאוד שקט וצמוד ל...,0,0,0,1,0,0,1,0,0,1,3500.0,3,3.042
8,כללי,פלורנטין,הרצל 87,3.0,0,800,0,0,12,100,100,0,בנסיבות משמחות מתפנה חדר בדירת שני שותפים בהרצ...,0,0,0,1,0,0,0,0,0,1,3150.0,6,2.879
9,גג/פנטהאוז,עג'מי,השקמה 5,4.0,2,115,0,0,12,NaN,NaN,2,יש מזגנים שלושה חדרי שינה וסלון גג ענק מרווחת ...,0,0,0,1,0,0,0,1,0,1,8800.0,9,5.212
